##### Using symbolic dynamics

In [551]:
import numpy as np
import sympy as sp
from sympy.matrices import Matrix
from sympy import Array

from sympy.interactive.printing import init_printing
init_printing(use_unicode=False, wrap_line=False, no_global=True)

def symbolic_dynamics_pendulum():
    """Symbolic dynamics for a pendulum

    Returns:
        sympy.Function: f(x.u)
        sympy.Function: df/dx
        sympy.Function: df/du
    """
    m, g, L, theta, theta_dot, u, dt = sp.symbols('m g L theta theta_dot u dt')

    # inputs and states
    inputs = Matrix([u])
    states = Matrix([theta, theta_dot])

    # dynamics for a pendulum of mass m and center of mass L
    # ----------------
    #        |\
    #        | \
    # \theta |->\
    #        |  []
    f = Matrix([theta_dot, (u-m*g*L*sp.sin(theta))/(m*L*L)])

    # discretize the system using euler integration
    f = states + f*dt

    # first derivatives wrt to (x, u)
    f_x = f.jacobian(states) # df/dx
    f_u = f.jacobian(inputs) # df/du

    # second derivatives wrt to (x, u)
    f_xx = Matrix([[sp.hessian(f[0], states)], [sp.hessian(f[1], states)]])
    f_uu = Matrix([sp.hessian(f[0], inputs), sp.hessian(f[1], inputs)])
    f_xu = f_u.jacobian(states)

    # symbols
    # print("f {} = \n{}".format(f.shape, f))
    # print("f_u {} = \n{}".format(f_u.shape, f_u))
    # print("f_x {} = \n{}".format(f_x.shape, f_x))
    # print("f_xx {} = \n{}".format(f_xx.shape, f_xx))
    # print("f_uu {} = \n{}".format(f_uu.shape, f_uu))
    # print("f_xu {} = \n{}".format(f_xu.shape, f_xu))

    # define parameters
    parameters = Matrix([m,g,L])

    # create lambdas
    arg = (states, inputs, dt, parameters)
    f_func = sp.lambdify(arg, f)
    f_x_func = sp.lambdify(arg, f_x)
    f_u_func = sp.lambdify(arg, f_u)
    f_xx_func = sp.lambdify(arg, f_xx)
    f_uu_func = sp.lambdify(arg, f_uu)
    f_xu_func = sp.lambdify(arg, f_xu)

    return (f_func, f_x_func, f_u_func, f_xx_func, f_uu_func, f_xu_func)

(f,f_x,f_u,f_xx, f_uu, f_xu) = symbolic_dynamics_pendulum()

# parameters
mass = 1
gravity = 9.8
pendulum_length = 1
parameters = np.array([mass, gravity, pendulum_length])
dt = 0.005

# current states and inputs
x = np.array([0.1, 0.1])
u = np.array([0.5])
n_states = len(x)
n_inputs = len(u)

f_v = f(x, u, dt, parameters)
f_x_v = f_x(x, u, dt, parameters)
f_u_v = f_u(x, u, dt, parameters)
f_xx_v = f_xx(x, u, dt, parameters)
f_uu_v = f_uu(x, u, dt, parameters)
f_xu_v = f_xu(x, u, dt, parameters)

print("f = \n{}".format(f_v))
print("f_x = \n{}".format(f_x_v))
print("f_u = \n{}".format(f_u_v))
print("f_xx = \n{}".format(f_xx_v))
print("f_uu = \n{}".format(f_uu_v))
print("f_xu = \n{}".format(f_xu_v))

f = 
[[0.1005    ]
 [0.09760816]]
f_x = 
[[ 1.         0.005    ]
 [-0.0487552  1.       ]]
f_u = 
[[0.   ]
 [0.005]]
f_xx = 
[[0.         0.        ]
 [0.         0.        ]
 [0.00489184 0.        ]
 [0.         0.        ]]
f_uu = 
[[0]
 [0]]
f_xu = 
[[0 0]
 [0 0]]


##### Using scipy

In [ ]:
from scipy.optimize import approx_fprime

def f(x, u, dt, parameters):
    m = parameters[0]
    g = parameters[1]
    L = parameters[2]
    return np.array([[dt*x[1] + x[0]], [x[1] + dt*(-L*g*m*np.sin(x[0]) + u)/(L**2*m)]])

def f_x(x, u, dt, parameters):
    m = parameters[0]
    g = parameters[1]
    L = parameters[2]
    return np.array([[1, dt], [-dt*g*np.cos(x[0])/L, 1]])

def f_u(x, u, dt, parameters):
    m = parameters[0]
    g = parameters[1]
    L = parameters[2]
    return np.array([[0], [dt/(L**2*m)]])

eps = np.sqrt(np.finfo(float).eps)
f_x_v = np.vstack([approx_fprime(x, lambda x: f(x, u, dt, parameters)[m], eps) for m in range(n_states)])
f_u_v = np.vstack([approx_fprime(u, lambda u: f(x, u, dt, parameters)[m], eps) for m in range(n_states)])
print("f_x {} = \n{}".format(f_x_v.shape, f_x_v))
print("f_u {} = \n{}".format(f_u_v.shape, f_u_v))

f_xx_v = np.array([
            [
                approx_fprime(x, lambda x: f_x(x, u, dt, parameters)[m, n], eps)
                for n in range(n_states)
            ]
            for m in range(n_states)
        ])
print("f_xx {} = \n{}".format(f_xx_v.shape, f_xx_v))

f_uu_v = np.array([
            [
                approx_fprime(u, lambda u: f_u(x, u, dt, parameters)[m, n], eps)
                for n in range(n_inputs)
            ]
            for m in range(n_states)
        ])
f_uu_v = np.squeeze(f_uu_v, axis=1)
print("f_uu {} = \n{}".format(f_uu_v.shape, f_uu_v))

f_xu_v = np.array([
            [
                approx_fprime(x, lambda x: f_u(x, u, dt, parameters)[m, n], eps)
                for n in range(n_inputs)
            ]
            for m in range(n_states)
        ])
f_xu_v = np.squeeze(f_xu_v, axis=1)
print("f_xu {} = \n{}".format(f_xu_v.shape, f_xu_v))

f_x (2, 2) = 
[[ 1.         0.005    ]
 [-0.0487552  1.       ]]
f_u (2, 1) = 
[[0.   ]
 [0.005]]
f_xx (2, 2, 2) = 
[[[0.         0.        ]
  [0.         0.        ]]

 [[0.00489184 0.        ]
  [0.         0.        ]]]
f_uu (2, 1) = 
[[0.]
 [0.]]
f_xu (2, 2) = 
[[0. 0.]
 [0. 0.]]


/tmp/ipykernel_4961/3972649313.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([[dt*x[1] + x[0]], [x[1] + dt*(-L*g*m*np.sin(x[0]) + u)/(L**2*m)]])


In [ ]:
import jax
import jax.numpy as jnp

m = 1
g = 9.8
L = 1
dt = 0.005

def f(x, u):
    return jnp.array([
        [dt*x[1] + x[0]],                                         # [dt*theta_dot + theta]
        [x[1] + dt*(-L*g*m*jnp.sin(x[0]) + u)/(L**2*m)]           # [theta_dot + dt*(-L*g*m*sin(theta) + u)/(L**2*m)]
        ])

x = jnp.array([0.1, 0.1])
u = jnp.array(0.5)

f_ = f(x, u)
print("f {} = \n{}".format(f_.shape, f_))

f_x = jnp.squeeze(jacfwd(f, 0)(x, u), axis=1)
print("f_x {} = \n{}".format(f_x.shape, f_x))

f_u = jacfwd(f, 1)(x, u)
print("f_u {} = \n{}".format(f_u.shape, f_u))

f_xx = jnp.squeeze(jacfwd(jacrev(f))(x, u), axis=1)
print("f_xx {} = \n{}".format(f_xx.shape, f_xx))

f_uu = jacfwd(jacrev(f, 1), 1)(x, u)
print("f_uu {} = \n{}".format(f_uu.shape, f_uu))

f_xu = jnp.squeeze(jacfwd(jacrev(f, 1), 0)(x, u), axis=1)
print("f_xu {} = \n{}".format(f_xu.shape, f_xu))

f (2, 1) = 
[[0.1005    ]
 [0.09760816]]
f_x (2, 2) = 
[[ 1.          0.005     ]
 [-0.04875521  1.        ]]
f_u (2, 1) = 
[[0.   ]
 [0.005]]
f_xx (2, 2, 2) = 
[[[0.         0.        ]
  [0.         0.        ]]

 [[0.00489184 0.        ]
  [0.         0.        ]]]
f_uu (2, 1) = 
[[0.]
 [0.]]
f_xu (2, 2) = 
[[0. 0.]
 [0. 0.]]
